In [1]:
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

aws_access_key = os.environ.get("AWS_ACCESS_KEY")
aws_secret_access_key = os.environ.get("AWS_SECRET_ACCESS_KEY")
aws_bucket = os.environ.get("AWS_BUCKET")
root_dir = os.environ.get("ROOT_DIR")

In [3]:
import boto3

In [4]:
# Initialize the session with authentication
session = boto3.Session(
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_access_key,
)
 
# Get the interface to AWS s3
#s3_client = session.client(
#    "s3", endpoint_url="https://s3.wasabisys.com"
#)

s3_resource = session.resource("s3", endpoint_url = 'https://s3.wasabisys.com')
s3_bucket = s3_resource.Bucket(aws_bucket)

In [10]:
paths = ('core/', 'geo-supplement/', 'monthly-patterns/normalization_stats', 
         'monthly-patterns/patterns/', 'monthly-patterns/patterns_backfill/2020/05/07/12/2019/',
         'monthly-patterns/patterns_backfill/2020/05/07/12/2020/','open-census-data/')

In [9]:
# local directory where we want to put all the data
local = os.path.join(root_dir,'data/raw')
n = len(local)
# print(local)
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(local):
    for file in f:
        files.append(os.path.join(r, file)[n+1:])
#for file in files:
#    print(file)
count = 0
for obj in s3_bucket.objects.all():
    count = count + 1
    file = obj.key
    if file.startswith(paths) and file not in files:
        print(file)
        filepath = os.path.join(local, file)
        directory = os.path.dirname(filepath)
        os.makedirs(directory, exist_ok=True)
        s3_bucket.download_file(file, filepath)

monthly-patterns/patterns_backfill/2020/05/07/12/2019/01/patterns-part3.csv.gz
monthly-patterns/patterns_backfill/2020/05/07/12/2019/01/patterns-part4.csv.gz
monthly-patterns/patterns_backfill/2020/05/07/12/2019/02/_SUCCESS
monthly-patterns/patterns_backfill/2020/05/07/12/2019/02/patterns-part1.csv.gz
monthly-patterns/patterns_backfill/2020/05/07/12/2019/02/patterns-part2.csv.gz
monthly-patterns/patterns_backfill/2020/05/07/12/2019/02/patterns-part3.csv.gz
monthly-patterns/patterns_backfill/2020/05/07/12/2019/02/patterns-part4.csv.gz
monthly-patterns/patterns_backfill/2020/05/07/12/2019/03/_SUCCESS
monthly-patterns/patterns_backfill/2020/05/07/12/2019/03/patterns-part1.csv.gz
monthly-patterns/patterns_backfill/2020/05/07/12/2019/03/patterns-part2.csv.gz
monthly-patterns/patterns_backfill/2020/05/07/12/2019/03/patterns-part3.csv.gz
monthly-patterns/patterns_backfill/2020/05/07/12/2019/03/patterns-part4.csv.gz
monthly-patterns/patterns_backfill/2020/05/07/12/2019/04/_SUCCESS
monthly-patt

In [7]:
for obj in s3_bucket.objects.all():
    print(obj.key)

2020-03-01-weekly-patterns.csv.gz
core-places-delivery/brand_info/2020/11/06/11/_SUCCESS
core-places-delivery/brand_info/2020/11/06/11/brand_info.csv
core-places-delivery/brand_info/2020/11/06/12/_SUCCESS
core-places-delivery/brand_info/2020/11/06/12/brand_info.csv
core-places-delivery/core_poi/2020/11/06/11/_SUCCESS
core-places-delivery/core_poi/2020/11/06/11/core_poi-part1.csv.gz
core-places-delivery/core_poi/2020/11/06/11/core_poi-part2.csv.gz
core-places-delivery/core_poi/2020/11/06/11/core_poi-part3.csv.gz
core-places-delivery/core_poi/2020/11/06/11/core_poi-part4.csv.gz
core-places-delivery/core_poi/2020/11/06/11/core_poi-part5.csv.gz
core-places-delivery/core_poi/2020/11/06/12/_SUCCESS
core-places-delivery/core_poi/2020/11/06/12/core_poi-part1.csv.gz
core-places-delivery/core_poi/2020/11/06/12/core_poi-part2.csv.gz
core-places-delivery/core_poi/2020/11/06/12/core_poi-part3.csv.gz
core-places-delivery/core_poi/2020/11/06/12/core_poi-part4.csv.gz
core-places-delivery/core_poi/2020

social-distancing/v2/2020/04/09/2020-04-09-social-distancing.csv.gz
social-distancing/v2/2020/04/10/2020-04-10-social-distancing.csv.gz
social-distancing/v2/2020/04/11/2020-04-11-social-distancing.csv.gz
social-distancing/v2/2020/04/12/2020-04-12-social-distancing.csv.gz
social-distancing/v2/2020/04/13/2020-04-13-social-distancing.csv.gz
social-distancing/v2/2020/04/14/2020-04-14-social-distancing.csv.gz
social-distancing/v2/2020/04/15/2020-04-15-social-distancing.csv.gz
social-distancing/v2/2020/04/16/2020-04-16-social-distancing.csv.gz
social-distancing/v2/2020/04/17/2020-04-17-social-distancing.csv.gz
social-distancing/v2/2020/04/18/2020-04-18-social-distancing.csv.gz
social-distancing/v2/2020/04/19/2020-04-19-social-distancing.csv.gz
social-distancing/v2/2020/04/20/2020-04-20-social-distancing.csv.gz
social-distancing/v2/2020/04/21/2020-04-21-social-distancing.csv.gz
social-distancing/v2/2020/04/22/2020-04-22-social-distancing.csv.gz
social-distancing/v2/2020/04/23/2020-04-23-socia